In [49]:
import numpy as np
x = 5
y = 5
pocatek = (0,0)
matrixDensity = 0.1

n = int(x * y * matrixDensity)
space = np.zeros((x*y), dtype=int)
rngX = np.random.choice(range(x*y), n, replace=False)
for i in rngX:
    space[i] = 1

space.reshape(x, y)



array([[0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])